In [ ]:
import pandas as pd
import datetime as dt
from statistics import mean

df = pd.read_csv('EVCS_edited.csv')

In [ ]:
df['PastWkOccRate'] = 0
df.rename(columns = {'Date': 'Day'}, inplace=True)
# df.head()

In [ ]:
df['FullDate'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
df.head()

In [ ]:
df_split = dict()
for station in df['Station_Name'].unique():
    df_split[station] = df[df['Station_Name'] == station].sort_values(['Year', 'Month', 'Day', 'Start_Hour']).reset_index()

In [ ]:
for name, station in df_split.items():
    for i in range(len(station)):
        if i == 0:
            station.at[i, 'Past24Avg'] = 0
        elif i < 24:
            station.at[i, 'Past24Avg'] = mean([station.at[j, 'OccupationRate'] for j in range(i-1, -1, -1)])
        else:
            station.at[i, 'Past24Avg'] = mean([station.at[j, 'OccupationRate'] for j in range(i-1, i-25, -1)])

In [ ]:
for name, station in df_split.items():
    
    # setting firstDay in station data
    firstDay = station.at[0, 'FullDate']
    
    for i, row in station.iterrows():
        hour = row['Start_Hour']

        # Getting start date for past week
        today = row['FullDate']
        
        # Skipping calculations if its first day
        if today == firstDay:
            station['PastWkOccRate'] = 0
            continue
            
        else:
            prevWk = [today - dt.timedelta(days=i) for i in range(1, 8)]
            
            prevWkDF = station[station['FullDate'].isin(prevWk)]
            # Creating mask for past 7 days for same hour
            mask = (prevWkDF['Start_Hour'] == hour)i
            prevWkDF = prevWkDF.loc[mask, 'OccupationRate']

            # Calculating average occupation for last 7 days
            avgOccRate = prevWkDF.mean()
            station.at[i, 'PastWkOccRate'] = avgOccRate

In [ ]:
del df

df = pd.concat([df_split[station] for station in df_split])

df.info()

In [ ]:
df.to_csv('EVCS_MA.csv')

old_df = pd.read_csv('EVCS_edited.csv')

old_df.info()